### Chatbot for game recommandation

In [1]:
import requests
import time

CLIENT_ID = "9yb5dk90lp8h1s2nt9eey4t2ppbjks"
ACCESS_TOKEN = "6wc6k971ez94isuroypcpk7inl1l49"
HEADERS = {
    "Client-ID": CLIENT_ID,
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

# Fonction pour récupérer tous les jeux en paginant
def get_all_games(limit_per_request=500):  
    all_games = []
    offset = 0  

    while True:
        query = f"""
        fields name, summary, genres.name, rating;
        sort rating desc;
        limit {limit_per_request};
        offset {offset};
        """

        response = requests.post("https://api.igdb.com/v4/games", headers=HEADERS, data=query)

        if response.status_code != 200:
            print(f"Erreur {response.status_code}: {response.text}")
            break  

        games = response.json()

        if not games:  
            break

        all_games.extend(games)
        offset += limit_per_request  
 

        time.sleep(0.5)  

    return all_games

# Récupération de tous les jeux
games = get_all_games()
 
import json
with open("igdb_games.json", "w", encoding="utf-8") as f:
    json.dump(games, f, ensure_ascii=False, indent=4)



🔄 Jeux récupérés : 500
🔄 Jeux récupérés : 1000
🔄 Jeux récupérés : 1500
🔄 Jeux récupérés : 2000
🔄 Jeux récupérés : 2500
🔄 Jeux récupérés : 3000
🔄 Jeux récupérés : 3500
🔄 Jeux récupérés : 4000
🔄 Jeux récupérés : 4500
🔄 Jeux récupérés : 5000
🔄 Jeux récupérés : 5500
🔄 Jeux récupérés : 6000
🔄 Jeux récupérés : 6500
🔄 Jeux récupérés : 7000
🔄 Jeux récupérés : 7500
🔄 Jeux récupérés : 8000
🔄 Jeux récupérés : 8500
🔄 Jeux récupérés : 9000
🔄 Jeux récupérés : 9500
🔄 Jeux récupérés : 10000
🔄 Jeux récupérés : 10500
🔄 Jeux récupérés : 11000
🔄 Jeux récupérés : 11500
🔄 Jeux récupérés : 12000
🔄 Jeux récupérés : 12500
🔄 Jeux récupérés : 13000
🔄 Jeux récupérés : 13500
🔄 Jeux récupérés : 14000
🔄 Jeux récupérés : 14500
🔄 Jeux récupérés : 15000
🔄 Jeux récupérés : 15500
🔄 Jeux récupérés : 16000
🔄 Jeux récupérés : 16500
🔄 Jeux récupérés : 17000
🔄 Jeux récupérés : 17500
🔄 Jeux récupérés : 18000
🔄 Jeux récupérés : 18500
🔄 Jeux récupérés : 19000
🔄 Jeux récupérés : 19500
🔄 Jeux récupérés : 20000
🔄 Jeux récupérés : 2

In [2]:
from sentence_transformers import SentenceTransformer
import json

embedding_model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./cache")

# Charger les jeux IGDB
with open("igdb_games.json", "r") as f:
    games_data = json.load(f)

for game in games_data:
    game["embedding"] = embedding_model.encode(game.get("summary", "No description")).tolist()  

# Sauvegarde en JSON
with open("igdb_games_embedding.json", "w") as f:
    json.dump(games_data, f, indent=4)  

print("Base de données des jeux enregistrée avec embeddings !")


Base de données des jeux enregistrée avec embeddings !


In [3]:
import numpy as np
from scipy.spatial.distance import cosine

# Charger les jeux IGDB
with open("igdb_games_embedding.json", "r") as f:
    games = json.load(f)

# Fonction de recommandation basée sur la similarité
def recommend_game(user_query):
    query_embedding = embedding_model.encode(user_query)

    similarities = [
        (game["name"], 1 - cosine(query_embedding, game["embedding"]))
        for game in games if "embedding" in game
    ]

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:3]

# Test
user_input = "Je cherche un RPG avec un bon scénario et des personnages profonds."
print("Jeux recommandés :", recommend_game(user_input))


Jeux recommandés : [('Argo Navis', 0.614671952143916), ('Bosquet', 0.5988493173857006), ('Dissimilar', 0.5884370430092204)]


In [4]:
!pip install langchain langchain_openai "pydantic<2" openai
!pip install --upgrade typing_extensions

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://p

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

def chatbot_response(user_query):
    recommendations = recommend_game(user_query)
    games_list = ", ".join([game[0] for game in recommendations])
    prompt = f"""
    L'utilisateur cherche un jeu correspondant à : "{user_query}".
    Voici les jeux recommandés : {games_list}.
    Formule une réponse naturelle et engageante pour lui donner ces recommandations.
    """

    
    llm = ChatOpenAI(
        openai_api_key="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",
        openai_api_base="https://openrouter.ai/api/v1",
        model_name="openchat/openchat-7b:free",
    )

    
    response = llm.invoke([HumanMessage(content=prompt)])

    return response.content


user_input = "Je veux un jeu avec un bon scénario et des choix impactants."

print(chatbot_response(user_input))


ImportError: cannot import name 'TypeIs' from 'typing_extensions' (/usr/local/lib/python3.11/dist-packages/typing_extensions.py)